In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:

import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import html
import numpy as np
import math
import re
import json
import random

import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from datetime import datetime
import webbrowser
import csv
from collections import defaultdict


 
options = Options()
elem_p_page = 25  ## Linkedin shows only 25 results per page, max 1000 in total
linkedin_max_pages = 40 ## max de linkedin

## base_search_page: https://www.linkedin.com/jobs/search/?keywords=developer%20python&location=Germany&f_TPR=r2592000

user_data= "your@email.com"
password = "your pass"

searchterm_org = '"data scientist"'
searchterms = searchterm_org.split()
searchterm = searchterms[0] + '"' + ' "and" ' + '"'+ searchterms[1]
searchlocation = "Denmark"
periode = "r2592000"  # the code for month
number_of_pages = 0
# Denmark, geoID 104514075, Ireland 104738515


joburl_base = "https://www.linkedin.com/jobs/view/"



geoid = {"Denmark":"104514075","Ireland":"104738515","Spain":"105646813",
"Austria":"103883259","Francia":"105015875","Italy":"103350119","Germany":"101282230",
"Netherlands":"102890719", "Belgium":"100565514", "Sweden":"105117694", "Norway":"103819153",
"Finland":"100456013","Estonia":"102974008","Czechia":"104508036","Hungary":"100288700", "Greece":"104677530",
"Malta":"100961908", "Reino Unido":"101165590", "Poland":"105072130", "Portugal":"100364837"
}

options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
 
## Login page
URL = "https://www.linkedin.com/login?emailAddress=&amp;fromSignIn=&amp;fromSignIn=true&amp"



headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }



def sleep_flex(start=4,end=45):
    rand_start = random.randint(start,end)
    rand_end = random.randint(start+3,end+3)
    if rand_end>rand_start:
        sleep_flexo = random.randint(rand_start,rand_end)
        return time.sleep(sleep_flexo)
    else:
        return sleep_flex(5,29)
    
  

## Connecting to URL page and doing the sign-in
driver.get(URL)




user_input = driver.find_element(By.ID, 'username')
user_input.click()
sleep_flex(3,6)
user_input.send_keys(user_data)
sleep_flex(2,7)
password_input = driver.find_element(By.ID, 'password')
sleep_flex(2,7)
password_input.click()
sleep_flex(3,8)
password_input.send_keys(password)
sleep_flex(2,8)

signin = driver.find_element(By.XPATH, '//*[@id="organic-div"]/form/div[3]/button').click()
sleep_flex(2,9)
job_search_main_page  = driver.find_element(By.XPATH, '//*[@id="global-nav"]/div/nav/ul/li[3]/a').click()

sleep_flex(4,10)
job_search_field= driver.find_element(By.XPATH,"//*[starts-with(@id, 'jobs-search-box-keyword-id')]")
job_search_location = driver.find_element(By.XPATH,"//*[starts-with(@id, 'jobs-search-box-location-id')]")


## Doing clicks and inputs in the jobsearch form
job_search_field.click()
sleep_flex(2,15)
job_search_field.send_keys(searchterm)
sleep_flex(3,8)
job_search_location.click()
sleep_flex(3,15)
job_search_location.send_keys(searchlocation)
sleep_flex(4,16)
job_search_location.send_keys(Keys.RETURN)
sleep_flex(1,12)


print ("---"*10)

## Get actual URL
get_url = driver.current_url
print("The current url is:"+str(get_url))
sleep_flex(1,12)
URL = str(get_url)




## Doing the Soup
html = driver.page_source
soup = bs(html, "html.parser")


print ("----"*15)




country_results = pd.DataFrame(columns=["country","searchterm","number_results","timestamp"])


try:
    ### Calculation of max pages, although linkedin permits showing only 1000 results = 40 pages max
    number_results = soup.find_all(class_="jobs-search-results-list__text display-flex t-12 t-black--light t-normal")
    

    for i in number_results:
    #  print(i.get_text(strip=True))
        print(i.get_text(strip=True))
        qty_results = int(i.get_text(strip=True).replace(",","").split(" ")[0])
        number_of_pages = math.ceil(qty_results/elem_p_page)
        timestamp= datetime.now()
        to_append= [searchlocation,searchterm_org,qty_results,timestamp]
        a_series = pd.Series(to_append, index = country_results.columns)
        country_results = country_results.append(a_series, ignore_index=True)
        

        if qty_results >= 1000:
            max_results = 1000
        else:
            max_results = qty_results

    # print(number_of_pages)

    print (max_results)


    if number_of_pages >= linkedin_max_pages: 
        number_of_pages = 40
    else:
        number_of_pages = number_of_pages


    ## Using of result numbers for iteration until 1000 results
    

    ## save country results to CSV
    country_results.to_csv('jobs_country.csv', mode='a', index=False, header=False, sep="|")

  

except:
    print ("error page_result")



#print ("Error retrieve number of search results")

def create_urls (searchterm:str,searchlocation:str,geoid:str,periode:str, max_results:int):

    """ Function for result page URL list creation of job-search results.
    Args:
    searchterm (str) : searchterm altered for using in URL building
    searchlocation (str) : country or place of the job search
    periode (str) : identifyer used by linkedin for the search periode
    geoid (str) : identifyer used by linkedin for the country
    max_results (int) : number of jobs found 

    Return:
    links_result_pages (list) : list of result pages 
    """

    searchterm = searchterm.replace(" ","%")
    location = searchlocation
    links_result_pages = []

    # base_search_page = f"https://www.linkedin.com/jobs/search/?keywords={searchterm}&location={location}&{periode}&start={num_results}"
    num_results = 0

    while num_results < max_results:

        url_to_scrap = f"https://www.linkedin.com/jobs/search/?f_TPR={periode}&geoId={geoid[searchlocation]}&keywords={searchterm}&location={location}&start={num_results}"
        num_results = num_results+25
        links_result_pages.append(url_to_scrap)

    print (links_result_pages)

    return links_result_pages


## Create Urls of the job results in order to iterate by clicking
lista_urls = create_urls(searchterm,searchlocation,geoid,periode, max_results)


### Log Print
print ("----"*15)
time.sleep(30)
get_url = driver.current_url
print("The current url is:"+str(get_url))


def perform_click  (number_of_pages:int, driver:object, joburl_base:str):
    """ Does clicking on the result pages and looks for the results found, creates a new
    url for each job in order to check details with the scraper

    Args:
    number_of_pages (int) : number of result pages (max. 40)
    driver (obj) : the selenium driver obj
    joburl_base : the base url for the job detail page

    Return:
    jobs_detail_url (list) : returns list with all the job-detail URLS
    
    """

    jobs_detail_url = []
    for e in range(1,number_of_pages+1):

            time.sleep(10)
            get_url = driver.current_url
            print("The current url is:"+str(get_url))
            

                ## Working example of clicking on page in pagination for scraping jobs
            try:
                    next_page_btn = driver.find_element(By.CSS_SELECTOR,f"li[data-test-pagination-page-btn='{e}']")
                    next_page_btn.click()
                    html = driver.page_source
                    soup = bs(html, "html.parser")
                    sleep_flex(5,12)


                    ## Linkedin shows 25 job offers per page
                    print ("---"*17)
                    try:

                        selector = soup.find_all("li",attrs={"data-occludable-job-id":True})
                        for i in selector:
                        # print (i)
                            try:
                           
                                jobs_detail_url.append(joburl_base+i["data-occludable-job-id"])
                                

                            except:
                                print("error data-occludable-job-id")

                    except:
                        print ("error customized attribute")


                    print ("----"*15)
            

            except:
                    print (f"error click on {e} page")

    
    return  jobs_detail_url    
                        



## Clicking for getting the job urls                       
jobs_detail_url  = perform_click(number_of_pages, driver,joburl_base)

## Log print
print(jobs_detail_url)
print(len(jobs_detail_url))


def scrap_page (lista_urls:list, searchterm_org:str, searchlocation:str, driver):
    """ Function for scraping a Linkedin Job offer page from a list of linkedin-job urls.
    returns a list of the pages scraped and appends a CSV filed

    Args:
    lista_urls (list) : List of urls for which the scraping should be done
    searchterm_org (str) : String of the original searchterm which is used as information in the csv
    searchlocation (str) : The place or country for which the search was performed
    
    Return:
    list_jobs (pd object) : List of the jobs scraped
    """

    lista_jobs = pd.DataFrame( columns = [
        'id',"country", 'searchterm', 'title', 'company','location',
        'time','workplace','details','joburl','timestamp'
    ])
    

    for i,e in enumerate(lista_urls):

        url = e
        sleep_flex(2,8)
   
        try:
            driver.get(url)
            content = driver.page_source
            soup1 = bs(content, "html5lib")

            ## Get actual URL
            get_url = driver.current_url
            print("The current url is:"+str(get_url))
            sleep_flex(1,5)
            ## Reading the soup of the elements found
            
            try:
               job_title = soup1.find(class_="t-24 t-bold jobs-unified-top-card__job-title")
               title = job_title.get_text(strip=True)
               print(title)

            except:
                print("error reading job_title")
                title = " "

            try:
                job_company= soup1.find(class_="jobs-unified-top-card__company-name")
                company = job_company.get_text(strip=True)
                print (company)
                
            except:
                print("error reading job company" )
                company = " "
            
            try:
                job_location= soup1.find(class_="jobs-unified-top-card__bullet")
                location = job_location.get_text(strip=True)
                print (location)
            except:
                print("error reading job location")
                location = " "

            try:
                job_workplace = soup1.find(class_="jobs-unified-top-card__workplace-type")
                workplace = job_workplace.get_text(strip=True)
                print (workplace)

            except:
                print("error reading workplace")
                workplace = " "
            
            try:
                job_time= soup1.find(class_="jobs-unified-top-card__posted-date")
                published = job_time.get_text(strip=True)
                print(published)

            except:
                print ("error job time")
                published = " "


            # Log
            print ("---"*16)
            print(job_title,job_company,job_location,job_workplace,job_time)
            print ("---"*16)


            try:
                job_details = soup1.find('div',class_="mt5 jobs-box__html-content t-14").get_text()
                print(job_details)
            
            except:
                job_details2 = soup1.find_all('div',class_="mt5 jobs-box__html-content t-14")
                print("log error details")

                for i,e in enumerate(job_details2):
                    job_details = e.get_text(strip=True)
                    print(job_details)  

            

            #### new row creation - and appending CSV
            timestamp= datetime.now()
            new_job = {"id":i,"country":searchlocation,"searchterm":searchterm_org.replace('"',''),"title":title,"company":company,
            "location":location,"time":published,"workplace":workplace, "details":job_details,"joburl":url, "timestamp":timestamp}
            
            row = pd.DataFrame(new_job, index=[0])
            lista_jobs = lista_jobs.append(new_job, ignore_index=True)
            row.to_csv('linkedin.csv', mode='a', index=False, header=False, sep="|")
            

        except:
            print("error scraping")


    return lista_jobs

    ## End of function returning values
    #return lista_jobs



jobs_db = scrap_page(jobs_detail_url,searchterm_org,searchlocation, driver)

print(jobs_db)


# Delay
time.sleep(50)
driver.close()






------------------------------
The current url is:https://www.linkedin.com/jobs/search/?geoId=104514075&keywords=%22data%22%20%22and%22%20%22scientist%22&location=Denmark
------------------------------------------------------------
149 results
149
['https://www.linkedin.com/jobs/search/?f_TPR=r2592000&geoId=104514075&keywords="data"%"and"%"scientist"&location=Denmark&start=0', 'https://www.linkedin.com/jobs/search/?f_TPR=r2592000&geoId=104514075&keywords="data"%"and"%"scientist"&location=Denmark&start=25', 'https://www.linkedin.com/jobs/search/?f_TPR=r2592000&geoId=104514075&keywords="data"%"and"%"scientist"&location=Denmark&start=50', 'https://www.linkedin.com/jobs/search/?f_TPR=r2592000&geoId=104514075&keywords="data"%"and"%"scientist"&location=Denmark&start=75', 'https://www.linkedin.com/jobs/search/?f_TPR=r2592000&geoId=104514075&keywords="data"%"and"%"scientist"&location=Denmark&start=100', 'https://www.linkedin.com/jobs/search/?f_TPR=r2592000&geoId=104514075&keywords="data"%"and

/tmp/ipykernel_11025/332051175.py:163: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_results = country_results.append(a_series, ignore_index=True)


The current url is:https://www.linkedin.com/jobs/search/?geoId=104514075&keywords=%22data%22%20%22and%22%20%22scientist%22&location=Denmark
The current url is:https://www.linkedin.com/jobs/search/?geoId=104514075&keywords=%22data%22%20%22and%22%20%22scientist%22&location=Denmark
---------------------------------------------------
------------------------------------------------------------
The current url is:https://www.linkedin.com/jobs/search/?geoId=104514075&keywords=%22data%22%20%22and%22%20%22scientist%22&location=Denmark
---------------------------------------------------
------------------------------------------------------------
The current url is:https://www.linkedin.com/jobs/search/?geoId=104514075&keywords=%22data%22%20%22and%22%20%22scientist%22&location=Denmark&start=25
---------------------------------------------------
------------------------------------------------------------
The current url is:https://www.linkedin.com/jobs/search/?geoId=104514075&keywords=%22data%22

/tmp/ipykernel_11025/332051175.py:416: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lista_jobs = lista_jobs.append(new_job, ignore_index=True)


The current url is:https://www.linkedin.com/jobs/view/3179866197/
Data Scientist / Data Manager with a flair for Engineering
the LEGO Group
Billund, South Denmark, Denmark
error reading workplace
40 minutes ago
------------------------------------------------
<h1 class="t-24 t-bold jobs-unified-top-card__job-title">Data Scientist / Data Manager with a flair for Engineering</h1> <span class="jobs-unified-top-card__company-name">
                  <a class="ember-view t-black t-normal" href="/company/lego-group/life/" id="ember26">
                    the LEGO Group
                  </a>
            </span> <span class="jobs-unified-top-card__bullet">
                Billund, South Denmark, Denmark
              </span> None <span class="jobs-unified-top-card__posted-date">
                40 minutes ago
              </span>
------------------------------------------------

          Apply your vision and skills to combine hands-on technical engineering, data analytics and system integ

...

/tmp/ipykernel_11025/332051175.py:416: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lista_jobs = lista_jobs.append(new_job, ignore_index=True)


The current url is:https://www.linkedin.com/jobs/view/3141494318/
Research Assistant for implementation and evaluation of a rapid microbiological detection method
DTU - Technical University of Denmark
Kongens Lyngby, Capital Region, Denmark
On-site
3 weeks ago
------------------------------------------------
<h1 class="t-24 t-bold jobs-unified-top-card__job-title">Research Assistant for implementation and evaluation of a rapid microbiological detection method</h1> <span class="jobs-unified-top-card__company-name">
                  <a class="ember-view t-black t-normal" href="/company/technical-university-of-denmark/life/" id="ember26">
                    DTU - Technical University of Denmark
                  </a>
            </span> <span class="jobs-unified-top-card__bullet">
                Kongens Lyngby, Capital Region, Denmark
              </span> <span class="jobs-unified-top-card__workplace-type">On-site</span> <span class="jobs-unified-top-card__posted-date">
              

/tmp/ipykernel_11025/332051175.py:416: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lista_jobs = lista_jobs.append(new_job, ignore_index=True)


The current url is:https://www.linkedin.com/jobs/view/3144346585/
Scientist for Raw Material to Oral Formulations
Novo Nordisk
Måløv, Capital Region, Denmark
On-site
3 weeks ago
------------------------------------------------
<h1 class="t-24 t-bold jobs-unified-top-card__job-title">Scientist for Raw Material to Oral Formulations</h1> <span class="jobs-unified-top-card__company-name">
                  <a class="ember-view t-black t-normal" href="/company/novo-nordisk/life/" id="ember26">
                    Novo Nordisk
                  </a>
            </span> <span class="jobs-unified-top-card__bullet">
                Måløv, Capital Region, Denmark
              </span> <span class="jobs-unified-top-card__workplace-type">On-site</span> <span class="jobs-unified-top-card__posted-date">
                3 weeks ago
              </span>
------------------------------------------------

          Are you passionate about optimising tablet manufacturing processes and investigating the 

/tmp/ipykernel_11025/332051175.py:416: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lista_jobs = lista_jobs.append(new_job, ignore_index=True)
